## Package loading

In [144]:
import requests
from scrapy import Selector
import pandas as pd
from fake_useragent import UserAgent
import numpy as np

## Data gathering

In [148]:
global asin
asin = "B00NJ2M43M"

In [10]:
class getReview():
    page = 1
    ua = UserAgent()
    headers = {
        'authority': 'www.amazon.it',
        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "user-agent": ua.random
    }
    post_data = {
        "sortBy": "recent",
        "reviewerType": "all_reviews",
        "formatType": "",
        "mediaType": "",
        "filterByStar": "",
        "filterByLanguage": "",
        "filterByKeyword": "",
        "shouldAppend": "undefined",
        "deviceType": "desktop",
        "canShowIntHeader": "undefined",
        "pageSize": "10",
        "asin": asin,
    }

    def getPLPage(self):

        total_review = []

        while True:

            self.post_data["pageNumber"]= self.page,
            self.post_data["reftag"] = f"cm_cr_getr_d_paging_btm_next_{self.page}",
            self.post_data["scope"] = f"reviewsAjax{self.page}",
            spiderurl = f'https://www.amazon.com/hz/reviews-render/ajax/reviews/get/ref=cm_cr_arp_d_paging_btm_next_{self.page}'
            res = self.getRes(spiderurl,self.headers,'',self.post_data,'POST')

            if res:
                res = res.content.decode('utf-8')
                contents = res.split('&&&')
                for content in contents:
                    infos = content.split('","')
                    info = infos[-1].replace('"]','').replace('\\n','').replace('\\','')
                    if 'data-hook="review"' in info:
                        sel = Selector(text=info)
                        data = {}
                        data['product_name'] = 'Sony ZX Series Wired On-Ear Headphones, White MDR-ZX110'
                        data['username'] = sel.xpath('//span[@class="a-profile-name"]/text()').extract_first() #username
                        data['verification'] = sel.xpath('//span[@data-hook="avp-badge"]/text()').extract_first() #verified purchase
                        data['point'] = sel.xpath('//span[@class="a-icon-alt"]/text()').extract_first() #rating
                        data['date'] = sel.xpath('//span[@data-hook="review-date"]/text()').extract_first() #date
                        data['review'] = sel.xpath('//a[@data-hook="review-title"]//span').extract_first() #review title
                        data['detail'] = sel.xpath('//span[@data-hook="review-body"]//span').extract_first() #review text
                        data['helpful'] = sel.xpath('//span[@data-hook="helpful-vote-statement"]').extract_first() #helpful
                        image = sel.xpath('div[@class="review-image-tile-section"]').extract_first()
                        data['image'] = image if image else "not image" #image
                        print('data + 1')
                        total_review.append(data)
                
            # Each page has 10 comments, 99 pages is 1000
            if self.page <= 99:
                print('Next Page')
                self.page += 1
            else:
                break

        df = pd.DataFrame(total_review)
        #df.to_csv('./review_database/demo.csv')
        print("Data Gathering Successful!")
 
    def getApiIp(self):
        api_url = 'https://www.amazon.com/Bose-Cancelling-Wireless-Bluetooth-Headphones/dp/B07Q4QK379?ref_=ast_slp_dp&th=1&psc=1'
        res = requests.get(api_url, timeout=5)
        try:
            if res.status_code == 200:
                api_data = res.json()['data'][0]
                proxies = {
                    'http': 'http://{}:{}'.format(api_data['ip'], api_data['port']),
                    'https': 'http://{}:{}'.format(api_data['ip'], api_data['port']),
                }
                print(proxies)
                return proxies
            else:
                print('Failed to acquire.')
        except:
            print('Failed to acquire.')
 
    def getRes(self,url,headers,proxies,post_data,method):
        if proxies:
            for i in range(3):
                try:
                    if method == 'POST':
                        res = requests.post(url,headers=headers,data=post_data,proxies=proxies)
                    else:
                        res = requests.get(url, headers=headers,proxies=proxies)
                    if res:
                        return res
                except:
                    print(f'Error on the {i+1}th request')
                else:
                    return None
        else:
            for i in range(3):
                proxies = self.getApiIp()
                try:
                    if method == 'POST':
                        res = requests.post(url, headers=headers, data=post_data, proxies=proxies)
                    else:
                        res = requests.get(url, headers=headers, proxies=proxies)
                    if res:
                        return res
                except:
                    print(f"Error on the {i+1}th request")
                else:
                    return None
 
if __name__ == '__main__':
    data = getReview().getPLPage()

Data Gathering Successful!
